# 1.0 데이터 불러오기

In [ ]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

In [ ]:
# 신문고
dfsin=pd.read_csv('./test/sinmungo_total.csv')[['text','agency','date','keyword']]
dfsin['year'] = pd.to_datetime(dfsin['date']).dt.year
print(dfsin.isnull().sum())
dfsin = dfsin.dropna(axis=0)
# dfsin

# # 네이버 다음
# # ['title', 'text', 'site', 'source', 'keyword', 'year']
# dfnd=pd.read_csv('./test/naver_daum_total.csv')
# print(dfnd.isnull().sum())
# dfnd = dfnd.dropna(axis=0)
# dfnd.isnull().sum()
# # dfnd

In [ ]:
# 텍스트 열을 기준으로 완전히 동일한 중복글의경우 제일 처음 등장한녀석만 살리고, 중복제거 (그래도 어느정도 중복내용존재)
# dfsin=dfsin.drop_duplicates(['text'])

# 1.1 데이터 속 인식 불가단어 미리보기
## Kiwi 이용해서 인식불가단어 검출 
### (ckonlpy에서 사용자단어 처리할때 사용할 것)

In [ ]:
dfsin['text'].apply(lambda x : x.upper().replace('[^ ㄱ-ㅣ가-힣|0-9|a-zA-Z]', " ")
                   ).to_csv('./test/kiwi_Word_Extract.txt', index=False, header=None, encoding='utf-8' )

In [ ]:
from kiwipiepy import Kiwi, Option
kiwi = Kiwi(num_workers=0, model_path='./', options=Option.LOAD_DEFAULT_DICTIONARY | Option.INTEGRATE_ALLOMORPH)

class ReaderExam:
    def __init__(self, filePath):
        self.file = open(filePath, encoding='utf-8')
    
    def read(self, id):
        if id == 0: self.file.seek(0)
        return self.file.readline()


In [ ]:
reader = ReaderExam('./test/kiwi_Word_Extract.txt')
kiwiextracted=kiwi.extract_words(reader.read, min_cnt=1000,max_word_len=10, min_score=1.5)
kiwiextracted

# 2.0 토큰화, 지역성

## 2.1 토큰화

### 2.1.1 사용자사전 추가

In [ ]:
def mkLocationList():
    newNouns=set()
    with open('./참고파일/지역명/철도역명.txt', encoding='utf-8') as f:
        for line in f:
            word=line.replace('\t','').replace('Noun','').strip('\n')
            if len(word)<2:continue
            newNouns.add(word)
    with open('./참고파일/지역명/법정동리명.txt', encoding='utf-8') as f:
        for line in f:
            word=line.replace('\t','').replace('Noun','').strip('\n')
            if len(word)<3 : continue
            newNouns.add(word)
    with open('./참고파일/지역명/시군구명.txt', encoding='utf-8') as f:    
        for line in f:
            word=line.replace('\t','').replace('Noun','').strip('\n')
            newNouns.add(word)

    return list(newNouns)

locations = mkLocationList()

def locationToken(tokens):
    for token in tokens:
        if token in locations:
            return 1
    return None
# locations = mkLocationList()
# locations

def mkNewWordList():
    newWordList=[]
    with open('./ckonlpy/사용자단어.txt' ,encoding='utf-8') as f:
        for line in f:
            newWordList.append(line)
    return newWordList

In [ ]:
from konlpy.tag import Okt;okt=Okt()
from ckonlpy.utils import load_wordset,load_ngram,load_replace_wordpair
from ckonlpy.tag import Twitter
from ckonlpy.tag import Postprocessor
twitter=Twitter()

사용자단어=mkNewWordList()
지역명=mkLocationList()
twitter.add_dictionary(사용자단어, 'Noun')
twitter.add_dictionary(지역명, 'Noun')
twitter.add_dictionary(['버스위치'], 'Noun')
 
# ngrams=[(('동탄','신도시'),'Noun'),(('무정','차'),'Noun')]
replace = load_replace_wordpair('./ckonlpy/replace.txt')
ngrams = load_ngram('./ckonlpy/ngram.txt')
stopwords = load_wordset('./ckonlpy/stopwords.txt')
passwords = load_wordset('./ckonlpy/passwords.txt')

postprocessor = Postprocessor(
    base_tagger = twitter, # base tagger
    stopwords = stopwords, # 해당 단어 거르기
#     passwords = passwords, # 해당 단어만 선택
    passtags = {'Noun','Alpha','Number' ,'Adjective' },#, 'Verb'}, # 해당 품사만 선택
    replace = replace, # 해당 단어 set 치환
    ngrams = ngrams    # 해당 복합 단어 set을 한 단어로 결합
)

def mkToken(sent):#ngrams에서 ' - ' 로 단어가 붙어나오는걸 띄어쓰기삭제
    tmp=re.sub(pattern='[^ ㄱ-ㅣ가-힣|0-9|a-zA-Z]+', repl=' ', string=sent).strip()
    return [s.replace(' - ','') for s,v in postprocessor.pos(tmp.upper()) if len(s)>1]

# sent='버스노선및 신호등 관련 민원'
print(mkToken(sent),'    지역성 : ',locationToken(mkToken(sent)))

In [ ]:
# import kss
# from hanspell import spell_checker
import re
sent='구조변경 차량 버스전용차로 운행 문의(경기남부)'
sent = re.sub(pattern='[^ ㄱ-ㅣ가-힣|0-9|a-zA-Z]+', repl=' ', string=sent).strip()
# print(spell_checker.check(sent))
# print(okt.pos(sent,stem=True))
print(kss.split_sentences(sent))
print(mkToken(sent))
# print(mkToken(sent),'    지역성 : ',locationToken(mkToken(sent)))

In [ ]:
# 텍스트를 토큰으로 분류
# dftest=dfsin.query('year=="2019"').reset_index()
dfsin['token']=dfsin['text'].apply(lambda x : mkToken(x))
dfsin[['text','token']]

In [ ]:
# text 를 기준으로 간단 중복제거했을때 -> 6098개로, 데이터가 10%만 남게됨
dfsin.drop_duplicates('text')

## 2.2 토큰의 지역분리(경기도 여부)

### 2.2.1 (신문고) 데이터 Agency로, 지역성

In [ ]:
# 관련기관으로 함께 작성해놓은것에서 지역성 text가 얼마나 연관있는지 테스트 -> 거의 ... 효과없음
def agencyLot(sent):
    locat=['시','군','구','도']
    if sent[-1] in locat:
        return False
    else:
        return True
tmp=dftemp[dftemp['agency'].apply(lambda x : agencyLot(x))]

In [ ]:
def newagency(x):
    tmp=x['agency']
    if agencyLot(tmp)==False : ##시.도 끝나는것 -> 그대로 시, 도 가져가기
        return tmp
    else : ##시 도 끝나지 않은것 ->  토큰 경기도 검사 -> 경기토큰이면 경기로 , 아니면 그대로 자기기관가져가기
        if isTokenGyeonggi(x['token']):
            return '경기도'
        else:
            return tmp


dftemp['agency']=dfsin['agency'].apply(lambda x:x.replace('교육청','').split()[0])
dftemp['agency']=dftemp[['agency','token']].apply(lambda row : newagency(row) , axis=1)
dftemp

In [ ]:
# dftemp['agency']=dftemp['agency'].apply(lambda x:x.replace('교육청','').split()[0])
dftemp.to_csv("./지역분류기관으로.csv",encoding='utf-8')# -> 신문고_지역분리완료

In [ ]:
#시 도 X
tmp[tmp['token'].apply(lambda x:isTokenGyeonggi(x))]

In [ ]:
dftemp[dftemp['agency'].apply(lambda x : agencyLot(x)==False)].query("agency=='강원도'")

### 2.2.2 경기도 토큰을 이용하여 경기도에 관한 내용들만 뽑기

In [ ]:
with open('./ckonlpy/경기_시군구키워드.txt', encoding='utf-8') as f:
    tmpset=set()
    for line in f:
        word=line.replace('\t','').replace('Noun','').strip('\n')
        if len(word)<2:continue
        
        words=word.split('.')
        if len(words)==1:
            tmpset.add(words[0])
        else:
            for word in words:
                tmpset.add(word)
#         print(word)
gyeonggiToken=list(tmpset)
def isAgencyGyeonggi(agency):
    if '경기' in agency:
        return True
    else:
        return False
    
def isTokenGyeonggi(tokens):
    for token in tokens:
        if token in gyeonggiToken:
            return True
    return False

In [ ]:
def 경기토큰2(tokens):
    경기토큰=['운정','운정신도시','하남','하남신도시','교산신도시','교산']
    for token in tokens:
        if token in 경기토큰:
            return False
    return True

# dfsin = dfsin[dfsin['token'].apply(lambda x : 경기토큰2(x))]


In [ ]:
#####경기도 관련기관으로 선택한것 거르기!
dfgyeonggi = dfsin[dfsin['agency'].apply(lambda x: isAgencyGyeonggi(x))]
dfgyeonggi 
# dfsin[ dfsin['token'].apply(lambda x: isTokenGyeonggi(x))]

In [ ]:
# 기관경기도 아닌데 토큰에 경기도 내 지역 나타난것
dfNotGG = dfsin[dfsin['agency'].apply(lambda x: isAgencyGyeonggi(x)) == False]
dfNotGG = dfNotGG[dfNotGG['token'].apply(lambda x: isTokenGyeonggi(x))]

In [ ]:
# 경기도를 관련기관으로 선택한것 + 그게아닌데 경기도 내 지역이 토큰검출된것
dftotalGG=pd.concat([dfgyeonggi,dfNotGG])#.to_csv('./경기도필터링.csv',sep=':',index=False, encoding='utf-8')
dftotalGG

## 2.3 토큰의 수단성

In [ ]:
stationlist=[]
with open('./ckonlpy/역명.txt' , 'r',encoding='utf-8') as f:
    lines=f.readlines()
    for line in lines:
        try:
            stationlist.append(line.split()[0]+'역')
        except:continue
stationlist

In [ ]:
waylist=['X',]
with open('./ckonlpy/수단.txt' , 'r',encoding='utf-8') as f:
    ways=f.readlines()
    for way in ways:
        waylist.append(way.replace('\n',''))
# waylist
stationlist=[]
with open('./ckonlpy/역명.txt' , 'r',encoding='utf-8') as f:
    lines=f.readlines()
    for line in lines:
        try:
            stationlist.append(line.split()[0]+'역')
        except:continue

metrolist='인동선,경부선,일산선,경인선,경원선,경춘선,분당선,신분당선,경의선,중앙선,신안산선,\
,안산선,과천선,수인선,경강선,ITX,공항철도,동해선,경전철,의정부경전철,용인경전철,김포골드라인\
,인천공항 자기부상열차,장항선'.split(',')

def whichWay(tokens):
    waydict={f'{i}':0 for i in waylist}
    #급행철도 , 광역급행철도 ,GTXABCD -> GTX
    #고속철도 -> KTX
    # 호선 -> 지하철
    
    for token in tokens:
        if '호선' in token or '지하철' in token or '경전철' in token or token in metrolist or token in stationlist:#지하철 1 증가
            waydict['지하철'] += 1
        elif '택시' in token:
            waydict['택시']+=1
        elif 'M버스' in token or '광역버스' in token or '광역급행버스' in token:
            waydict['광역버스']+=1.2
        elif '시내버스' in token:
            waydict['시내버스']+=1.2
        elif '고속버스' in token:
            waydict['고속버스']+=1.2
        elif '마을버스' in token:
            waydict['마을버스']+=1.3
        elif '버스' in token or 'BUS' in token or '번' in token or '여객' in token:
            waydict['버스']+=1
        elif '노선' in token or '배차간격' in token : waydict['버스']+=0.6
        elif 'KTX' in token:
            waydict['KTX']+=2
        elif 'SRT' in token:
            waydict['SRT']+=2
        elif '트램' in token:
            waydict['트램']+=1
        elif '광역급행철도' in token or '급행철도' in token or 'GTX' in token:
            waydict['GTX'] +=1.5
        elif '철도'in token or '도시철도' in token or '철도' == token or '광역철도' in token:
            waydict['철도'] +=1
    
    return sorted(waydict, key=lambda k : waydict[k] , reverse=True)[0]
            
whichWay(['안산선', '남양', '향남', '노선연장', '요구'])

In [ ]:
# dftotalGG['수단']=dftotalGG['token'].apply(lambda x : whichWay(x))
dftotalGG.query("수단=='택시'").head(55)

In [ ]:
# dftotalGG.to_csv('./경기도필터링_수단추가.csv',encoding='utf-8')

## 2.4 __ 11.19 겸 엑셀토큰합치는과정

In [ ]:
aadfhk=pd.read_csv('./20201205 지역재처리와수단재처리(전국)_겸.csv',encoding='utf-8')
aadfhk

In [ ]:
def rowmerge(x):
    tmp=[]
    xs=x.dropna()
    tmp.extend(xs)
    return tmp

cols=['수단', '노선', '정류장', '차량', '운전기사', '차고지', '버스전용차로', '광역', '환승', '배차간격', \
      '신설', '개선', '연장', '유지', '추가', '거부', '확정', '불친절', '안내', '오류', '시간', '사고', '요금',
       '안전', '친절', '자격요건', '서비스', '코로나', '교통약자']

dfhk=pd.read_csv('./지역완료,겸토큰하나로.csv',encoding='utf-8')
# dfhk['hktoken']=dfhk[cols].apply(lambda row : rowmerge(row) , axis=1)
# dfhk['mtoken']=dfhk[['token','hktoken']].apply(lambda x : hktokenmerge(x) , axis=1)
# dfhk=dfhk.drop([cols],axis=1)

In [ ]:
from ast import literal_eval
import warnings
dfhk['token']=dfhk['token'].apply(lambda x : literal_eval(x))
dfhk['hktoken']=dfhk['hktoken'].apply(lambda x : literal_eval(x))

In [ ]:
# dfsin['token']=dfsin['text'].apply(lambda x : mkToken(x.upper()))
dfhk.query("year==2020")

In [ ]:
dftest=dfhk[['text','year','token','hktoken']]

In [ ]:
# csv에서 읽었을때 token 행이 그냥 문자열로 인식 -> 리스트로 바꿔주는 코드
from ast import literal_eval
def hktokenmerge(x):
    tmp=[]
    tmp.extend(x.values[0]) #csv에서 리스트 불러오면 str로 그냥 읽혀버려서 그걸 리스트화함
    tmp.extend(x.values[1])
    return list(set(tmp))
dfhk['totaltokens']=dfhk[['token','hktoken']].apply(lambda x : hktokenmerge(x) , axis=1)

###  2.4.1 기존의 신문고 데이터와 합쳐보기 (테스트)
####  신문고데이터와 합치면서 중복을 제거해서 진행해본다 
####  (완벽한 중복 제거는 거의 불가하니 python의 기본 내장함수 수준으로만 중복제거 )

In [ ]:
## 기존의 신문고 데이터와 합치는부분,  
dftmp=pd.merge(dfsin,dftest.drop('token',axis=1),on='text').drop_duplicates('text')
dftmp["mtoken"]=dftmp['token']+dftmp['hktoken']
# dftmp=dftmp.drop(['token'],axis=1)
dftmp

## 2.5 __ 2019년과 2020년으로 분류하여 진행

In [ ]:
df=dftest
df

In [ ]:
## 2020년 쿼리로 추출하고, 네트워크까지 1123
df20=df.query('year==2020')


In [ ]:
## 2019년만 추출해서 네트워크 진행 1124_0019
df19 = df.query('year==2019')

In [ ]:
dfsin['agency'].unique()

In [ ]:
dfsin['agency'].unique()

In [ ]:
## 시.도 별 데이터 개수 뽑기
# dfhk.groupby(['agency','year']).describe().to_csv('./2019,2020시도별개수.csv')

In [ ]:
dfhk

In [ ]:
##겸토큰 하나로 합친거 여기서 dfsin으로 다른이름 지정!

dfsin=dfhk.query("year=='2020' and agency=='충청북도'")

# 3.0 워드클라우드

In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt
from collections import Counter

In [ ]:
#1차원 리스트로 풀어주기 ->  For Word Counting
texts_1d=[]
for i in dfsin['totaltokens']:    
# for i in dfsin['hktoken']:    
#     texts_1d.extend(changeKeyword(i))
        texts_1d.extend(i)

texts_1d

In [ ]:
# count = Counter(texts)
count = Counter(texts_1d)
tags = {}
for n, c in count.most_common(75):
    tags[n] = c

wc = WordCloud(font_path='c:/Windows/Fonts/malgun.ttf',
               width=800, height=600, scale=2.0, max_font_size=200, background_color="#ffffff")
#plt.rc('font', family='Nanum Gothic')

gen = wc.generate_from_frequencies(tags)
plt.figure()
plt.imshow(gen, interpolation='bilinear')
plt.show()

wc.to_file('./png/신문고_2020_충청북도_토탈토큰.png')

#plt.close()


# 4.0 Networkx__연관성분석

In [ ]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 
warnings.filterwarnings("ignore", category=UserWarning) 
import networkx as nx

#
## 여기서 쓰인 dataset 이 끝까지 쓰임!!
#

In [ ]:
def changeKeyword(tokens):##대전에서 버스위치가 스위치로 토큰화된부분 처리
    tmp=[]
    for token in tokens:
        tmp.append(token.replace('정보사','정보사이트'))
    return tmp
dataset=[]
for i in dfsin['totaltokens']:
# for i in dfsin['hktoken']:    
#     dataset.append(i)
    dataset.append(changeKeyword(i))
# dataset

In [ ]:
len(dataset)

In [ ]:
from apyori import apriori
# # Original 코드
# from apyori import apriori
# 연관 분석을 수행합니다.
# results = list(apriori(dataset,
#     min_support=0.06,
#     min_confidence=0.05,
#     min_lift=1.0,
#     max_length=2))
# print(results)

# # 연관 분석을 수행합니다.
results = list(apriori(dataset,
    min_support=0.06,
    min_confidence=0.8,
    min_lift=0.5,
    max_length=2))
# print(results)


In [ ]:
from collections import Counter
# 데이터 프레임 형태로 정리합니다.
columns = ['source', 'target', 'support']
network_df = pd.DataFrame(columns=columns)

### Apriori 알고리즘을, Dataframe으로 변환
# 규칙의 조건절을 source, 결과절을 target, 지지도를 support 라는 데이터 프레임의 피처로 변환합니다.
for result in results:
    if len(result.items) == 2:
        items = [x for x in result.items]
        row = [items[0], items[1], result.support]
        series = pd.Series(row, index=network_df.columns)
        network_df = network_df.append(series, ignore_index=True)

# print(network_df)

#### Node 사이즈를 만들어주기위한 부분
dataset1d=[]

for i in dataset:
    dataset1d.extend(i)

count = Counter(dataset1d)

node_df = pd.DataFrame(count.items(), columns=['node', 'nodesize'])
#원본
# node_df = node_df[node_df['nodesize'] >= 5] # 시각화의 편의를 위해 ‘nodesize’ 5 이하는 제거합니다.
# 중복ON,전국,통합년도
# node_df = node_df[node_df['nodesize'] >= len(dataset)/12] 
node_df = node_df[node_df['nodesize'] >= 9] 

# 시각화의 편의를 위해 ‘nodesize’ 5 이하는 제거합니다.
# node_df = node_df[node_df['nodesize']>=2] ##데이터 30~40개일때
print(node_df.head())

In [ ]:
import networkx as nx
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
# import numpy as np

plt.figure(figsize=(15,15))

# networkx 그래프 객체를 생성합니다.
G = nx.Graph()

# pr=nx.pageranke(G)

# node_df의 키워드 빈도수를 데이터로 하여, 네트워크 그래프의 ‘노드’역할을 하는 원을 생성합니다.
for index, row in node_df.iterrows():
    G.add_node(row['node'], nodesize=row['nodesize'])

# network_df의 연관 분석 데이터를 기반으로, 네트워크 그래프의 ‘관계’역할을 하는 선을 생성합니다.
for index, row in network_df.iterrows():
    G.add_weighted_edges_from([(row['source'], row['target'], row['support'])])

# 그래프 디자인과 관련된 파라미터를 설정합니다. 
# # original
# pos = nx.spring_layout(G, k=0.6, iterations=50)
# sizes = [G.nodes[node]['nodesize']*30 for node in G]
# #
# #원형
# pos = nx.spring_layout(G, k=1.7, iterations=50)#2019

pos = nx.spring_layout(G, k=3 , iterations=30)#2020
##kameda 써보기
# pos = nx.kamada_kawai_layout(G)
# #
# pos=nx.planar_layout(G)
# pos=nx.random_layout(G)
ncolor2020='#BEEBC8'
ncolor2019='#C9D7F5'

sizes = [G.nodes[node]['nodesize']*len(dataset)*1 for node in G] #사이즈
# sizes = [G.nodes[node]['nodesize']*0.9 for node in G] #사이즈

# nx.draw(G, pos=pos, node_size=sizes, node_color=ncolor2019)
nx.draw(G, pos=pos, node_size=sizes, node_color=ncolor2020)

nx.draw_networkx_labels(G, pos=pos, font_family="Malgun Gothic", font_size=20)

# kamada_kawai 써보기!
# nx.kamada_kawai_layout(G)

# 그래프를 출력합니다.
ax = plt.gca()
plt.show()


In [ ]:
## 4.1 네트워크 분석된 것의 키워드에 대한 연관성 추출
def mkNetworkWeight(network,keyword):
    forkey=pd.DataFrame(network[keyword].items(),columns=['keyword','Weight'])#.sort_values(by=['Weight'])
    forkey['Weight']=forkey['keyword'].apply(lambda x: network[keyword][x]['weight'])
    return forkey.sort_values(by=['Weight'],ascending=False)

# 노선,차량,정류장,운전기사 + 수단들

mkNetworkWeight(G,'개선')

In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

path = 'c:/Windows/Fonts/malgun.ttf'
font_name = fm.FontProperties(fname=path, size=50).get_name()
print(font_name)
plt.rc('font', family=font_name)
mpl.rcParams['axes.unicode_minus'] = False

## 4.1 네트워크 분석된 것의 키워드에 대한 연관성 추출

In [ ]:
G['노선']

# 6.0.1 LDA 구축
#### Mallet LDA vs Gensim LDA -> 정확한 기준이 없이, 둘 중 더 좋은 토픽을 제공하는것을 사용하면됨

In [ ]:

# data_word=[]
# for i in ['hktoken']:
#     data_word.append(i)
# data_word

## 6.1 Mallet__LDA

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from gensim import corpora, models
import gensim

id2word=corpora.Dictionary(dataset)           ##  Dictionary 로 쓰이는 부분 (DF_DICT)
id2word.filter_extremes(no_below = 10)        #   20회 이하로 등장한 단어는 삭제
texts = dataset                               ##  Texts로 쓰이는 부분  (tokenized_data)

corpus=[id2word.doc2bow(text) for text in texts] ## corpus로 쓰이는 부분 (DF_CORPUS)
mallet_path = '/Mallet/bin/mallet' 

In [ ]:
ldamallet = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=10, id2word=id2word)

In [ ]:
from tqdm import tqdm 
import re 
from gensim.models.ldamodel import LdaModel 
from gensim.models.callbacks import CoherenceMetric 
from gensim.models.callbacks import PerplexityMetric 
import logging 
import pickle 
import pyLDAvis.gensim 
from gensim.models.coherencemodel import CoherenceModel 

#### Mallet
def compute_coherence_values_Mallet(dictionary, corpus, texts, limit, start=4, step=2):
    coherence_values = []
    model_list = []
    for num_topics in tqdm(range(start, limit, step)):
#         print(f'{num_topics} 번째 토픽 검사중')
        model = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=num_topics, id2word=id2word)
        model_list.append(model)
        coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())
    return model_list, coherence_values

#### Gensim LDA
def compute_coherence_values(dictionary,corpus,texts,limit,start=5,step=2):
    coherence_values=[]
    model_list=[]
    for num_topics in tqdm(range(start,limit,step)):
#         print(f'{num_topics} 번째 토픽 검사중')
        model=LdaModel(corpus=corpus, id2word=dictionary, num_topics=num_topics)
        model_list.append(model)
        coherencemodel=CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())
    return model_list, coherence_values

def find_optimal_number_of_topics(ismallet, dictionary, corpus, texts, start=5, limit=21, step=2): 
    #토픽의 개수를 2~40개사이로 6step에 걸쳐서 나누어 진행
    
    if ismallet==0:# 일반 Gensim LDA 
        model_list, coherence_values = compute_coherence_values(dictionary=dictionary, corpus=corpus, texts=texts, start=start, limit=limit, step=step) 
    else:
        model_list, coherence_values = compute_coherence_values_Mallet(dictionary=dictionary, corpus=corpus, texts=texts, start=start, limit=limit, step=step) 
    
    x = range(start, limit, step) 
    topic_num = 0
    count = 0
    max_coherence = 0
    for m, cv in zip(x, coherence_values):
        print("Num Topics =", m, " has Coherence Value of", round(cv,5))
        coherence = cv
        if coherence >= max_coherence:
            max_coherence = coherence
            topic_num = m
            model_list_num = count   
        count = count+1
    optimal_model = model_list[model_list_num]
    model_topics = optimal_model.show_topics(formatted=False)
    plt.plot(x, coherence_values) 
    plt.xlabel("Num Topics") 
    plt.ylabel("Coherence score") 
    plt.legend(("coherence_values"), loc='best') 
    plt.show()
    return model_list, optimal_model

# # Can take a long time to run.
# model_list, coherence_values = tqdm(compute_coherence_values(dictionary=id2word, 
#                                                         corpus=corpus, texts=texts, start=4, limit=21, step=2))

# limit=21; start=4; step=2;
# x = range(start, limit, step)
# topic_num = 0
# count = 0
# max_coherence = 0
# for m, cv in zip(x, coherence_values):
#     print("Num Topics =", m, " has Coherence Value of", cv)
#     coherence = cv
#     if coherence >= max_coherence:
#         max_coherence = coherence
#         topic_num = m
#         model_list_num = count   
#     count = count+1

        
# # Select the model and print the topics
# optimal_model = model_list[model_list_num]
# model_topics = optimal_model.show_topics(formatted=False)
# #print(optimal_model.print_topics(num_words=10))

In [ ]:
# gensim_model_list, gensim_optimal_model = find_optimal_number_of_topics(0,id2word, corpus, texts, start=5, limit=21, step=2)
mallet_model_list, mallet_optimal_model = find_optimal_number_of_topics(1,id2word, corpus, texts, start=5, limit=21, step=2)

In [ ]:
gensim_model_list, gensim_optimal_model = find_optimal_number_of_topics(0,id2word, corpus, texts, start=5, limit=21, step=2)

In [ ]:
# def find_optimal_number_of_topics(ismallet, dictionary, corpus, texts, start=4, limit=21, step=2): 
#     #토픽의 개수를 2~40개사이로 6step에 걸쳐서 나누어 진행
    
#     if ismallet==0:# 일반 Gensim LDA 
#         model_list, coherence_values = compute_coherence_values(dictionary=dictionary, corpus=corpus, texts=texts, start=start, limit=limit, step=step) 
#     else:
#         model_list, coherence_values = compute_coherence_values_Mallet(dictionary=dictionary, corpus=corpus, texts=texts, start=start, limit=limit, step=step) 
    
#     x = range(start, limit, step) 
#     topic_num = 0
#     count = 0
#     max_coherence = 0
#     for m, cv in zip(x, coherence_values):
#         print("Num Topics =", m, " has Coherence Value of", round(cv,5))
#         coherence = cv
#         if coherence >= max_coherence:
#             max_coherence = coherence
#             topic_num = m
#             model_list_num = count   
#         count = count+1
#     optimal_model = model_list[model_list_num]
#     model_topics = optimal_model.show_topics(formatted=False)
#     plt.plot(x, coherence_values) 
#     plt.xlabel("Num Topics") 
#     plt.ylabel("Coherence score") 
#     plt.legend(("coherence_values"), loc='best') 
#     plt.show()
#     return model_list, optimal_model

# model_list, optimal_model = find_optimal_number_of_topics(1,id2word, corpus, texts, start=4, limit=21, step=2)

## 6.3 LDA의 토픽 모델링 후 시각화 및 Dataframe으로 뽑아내기

## 6.3.1 Mallet의 시각화

In [ ]:
import numpy as np
from gensim.models import LdaModel

def mallet_to_lda(optimal_model):
    model_gensim = LdaModel(
        id2word=optimal_model.id2word, num_topics=optimal_model.num_topics,
        alpha=optimal_model.alpha, eta=0, iterations=1000,
        gamma_threshold=0.001,
        dtype=np.float32
    )
    model_gensim.sync_state()
    model_gensim.state.sstats = optimal_model.wordtopics
    return model_gensim

model = mallet_to_lda(mallet_model_list[5])

import pyLDAvis
import pyLDAvis.gensim

pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(model, corpus, id2word)
pyLDAvis.save_html(vis, './lda/신문고_Mallet_LDA_20xx_NOT경기topic.html') 
vis


## 6.3.2 Gensim LDA 의 시각화

In [ ]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(gensim_optimal_model, corpus, id2word)
pyLDAvis.save_html(vis, './lda/신문고_LDA_20xx_NOT경기topic.html') 
vis


### https://projector.tensorflow.org/ 에 시각화하기!

In [ ]:
# https://projector.tensorflow.org/ 에 시각화하기!
filename="신문고_LDA_20xx"
model.wv.save_word2vec_format(f'./lda/{filename}') # 모델 저장
#!python -m gensim.scripts.word2vec2tensor --input f'./lda/{filename}' --output f'./lda/{filename}'
!python -m gensim.scripts.word2vec2tensor --input ./lda/신문고_LDA_20xx --output ./lda/신문고_LDA_20xx

### gensim_optimal_model 구분 잘하기 

## texts구분잘하기~!

In [ ]:
#시작 전 세팅
# optimal_model = mallet_optimal_model
optimal_model = gensim_optimal_model

## texts 에 원본 텍스트가 들어가야한다
texts = dfsin['text']
texts=texts.reset_index()['text']

# dataset=dfsin['text']
# corpus=corpus
# texts=dateset

In [ ]:
def format_topics_sentences(ldamodel=optimal_model, corpus=corpus, texts=texts):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    #ldamodel[corpus]: lda_model에 corpus를 넣어 각 토픽 당 확률을 알 수 있음
    for i, row in tqdm(enumerate(ldamodel[corpus])):
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num,topn=10)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']
    print(type(sent_topics_df))

    # Add original text to the end of the output
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
#     sent_topics_df = pd.concat([sent_topics_df,texts], axis=1)
    return(sent_topics_df)


df_topic_sents_keywords = format_topics_sentences(ldamodel=optimal_model, corpus=corpus, texts=texts)

In [ ]:
df_topic_sents_keywords

In [ ]:
# Format
df_dominant_topic = df_topic_sents_keywords#
df_dominant_topic.columns = [
    'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']
# Show
df_dominant_topic

In [ ]:
# Group top 5 sentences under each topic
sent_topics_sorteddf_mallet = pd.DataFrame()

sent_topics_outdf_grpd = df_topic_sents_keywords.groupby('Dominant_Topic')

for i, grp in sent_topics_outdf_grpd:
    sent_topics_sorteddf_mallet = pd.concat([sent_topics_sorteddf_mallet, 
                                             grp.sort_values(['Topic_Perc_Contrib'], ascending=[0]).head(1)], 
                                            axis=0)

# Reset Index    
sent_topics_sorteddf_mallet.reset_index(drop=True, inplace=True)


topic_counts = df_topic_sents_keywords['Dominant_Topic'].value_counts()
topic_counts.sort_index(inplace=True)

topic_contribution = round(topic_counts/topic_counts.sum(), 4)
topic_contribution

lda_inform = pd.concat([sent_topics_sorteddf_mallet, topic_counts, topic_contribution], axis=1)
lda_inform.columns=["Topic_Num", "Topic_Perc_Contrib", "Keywords", "Text","Num_Documents", "Perc_Documents"]
lda_inform = lda_inform[["Topic_Num","Keywords","Num_Documents","Perc_Documents"]]
lda_inform
# #lda_inform.Topic_Num = lda_inform.Topic_Num.astype(int)
# lda_inform['Topic_Num'] =lda_inform['Topic_Num'] +1
# lda_inform.Topic_Num = lda_inform.Topic_Num.astype(str)
# lda_inform['Topic_Num'] =lda_inform['Topic_Num'].str.split('.').str[0]
# df_topic_tweet['Dominant_Topic'] =df_topic_tweet['Dominant_Topic'] +1
# df_topic_tweet.Dominant_Topic = df_topic_tweet.Dominant_Topic.astype(str)
# df_topic_tweet['Dominant_Topic'] =df_topic_tweet['Dominant_Topic'].str.split('.').str[0]
 

In [ ]:
sent_topics_sorteddf = pd.DataFrame()
sent_topics_outdf_grpd = df_topic_sents_keywords.groupby('Dominant_Topic')
for i, grp in sent_topics_outdf_grpd:
    sent_topics_sorteddf = pd.concat([sent_topics_sorteddf, 
                                             grp.sort_values(['Topic_Perc_Contrib'], ascending=[0]).head(1)], 
                                            axis=0)

# Reset Index    
sent_topics_sorteddf.reset_index(drop=True, inplace=True)

# Format
sent_topics_sorteddf.columns = ['Topic_Num', "Topic_Perc_Contrib", "Keywords", "Text"]

# Show
sent_topics_sorteddf